In [21]:
%matplotlib inline
import pandas as pd
import datetime as datetime
from dateutil.parser import parse
import numpy as np
import warnings
import datetime as dt
warnings.filterwarnings('ignore')

In [2]:
# Carga de archivos
post_educacion = pd.read_csv('data/fiuba_1_postulantes_educacion.csv')
post_gen_edad = pd.read_csv('data/fiuba_2_postulantes_genero_y_edad.csv')
vistas = pd.read_csv('data/fiuba_3_vistas.csv')
postulaciones = pd.read_csv('data/fiuba_4_postulaciones.csv')
avisos = pd.read_csv('data/fiuba_5_avisos_online.csv')
avisos_detalles = pd.read_csv('data/fiuba_6_avisos_detalle.csv')


In [3]:
# Cambiar el tipo de dato de la fecha de postulacion.
postulaciones[['fechapostulacion']] = postulaciones[['fechapostulacion']].apply(pd.to_datetime)


In [4]:
# Asigno puntajes a los diferentes niveles educativos encontrados para poder 
# obtener asi la educacion mas valiosa de cada postulante.
valores_educacion = [{'nombre': 'Otro', 'estado': 'Abandonado',    'valor_educacion': 1    },
    {'nombre': 'Otro',                  'estado': 'En Curso',    'valor_educacion': 2    },
    {'nombre': 'Otro',                  'estado': 'Graduado',    'valor_educacion': 3    },
    {'nombre': 'Secundario',            'estado': 'Abandonado',    'valor_educacion': 4    },
    {'nombre': 'Secundario',            'estado': 'En Curso',    'valor_educacion': 5    },
    {'nombre': 'Secundario',        'estado': 'Graduado',    'valor_educacion': 6    },
    {'nombre': 'Terciario/Técnico', 'estado': 'Abandonado',    'valor_educacion': 7    },
    {'nombre': 'Universitario',     'estado': 'Abandonado',    'valor_educacion': 8    },
    {'nombre': 'Terciario/Técnico', 'estado': 'En Curso',    'valor_educacion': 9    },
    {'nombre': 'Universitario',     'estado': 'En Curso',    'valor_educacion': 10    },
    {'nombre': 'Terciario/Técnico', 'estado': 'Graduado',    'valor_educacion': 11    },
    {'nombre': 'Universitario',     'estado': 'Graduado',    'valor_educacion': 12    },
    {'nombre': 'Posgrado',          'estado': 'Abandonado',    'valor_educacion': 13    },
    {'nombre': 'Master',            'estado': 'Abandonado',    'valor_educacion': 14    },
    {'nombre': 'Posgrado',          'estado': 'En Curso',    'valor_educacion': 15    },
    {'nombre': 'Master',            'estado': 'En Curso',    'valor_educacion': 16    },
    {'nombre': 'Posgrado',          'estado': 'Graduado',    'valor_educacion': 17    },
    {'nombre': 'Master',            'estado': 'Graduado',    'valor_educacion': 18    },
    {'nombre': 'Doctorado',         'estado': 'Abandonado',    'valor_educacion': 19    },
    {'nombre': 'Doctorado',         'estado': 'En Curso',    'valor_educacion': 20    },
    {'nombre': 'Doctorado',         'estado': 'Graduado',    'valor_educacion': 21 }]
valores_educacion = pd.DataFrame(valores_educacion)

In [5]:
# Hago el join para aplicar puntajes a las educaciones de los postulantes.
post_educacion = pd.merge(post_educacion, valores_educacion,  how='inner', \
                          left_on=['nombre','estado'], right_on = ['nombre','estado'])

In [6]:
# Busco la maxima educacion que tiene cada postulante.
max_post_educacion = post_educacion.groupby(['idpostulante'])['valor_educacion'].max().reset_index()
max_post_educacion.count()

idpostulante       188752
valor_educacion    188752
dtype: int64

In [7]:
# Hago el join para filtrar todas las educaciones que no me interesan, dejando solo la maxima educacion.
post_educacion = pd.merge(post_educacion, max_post_educacion,  how='inner', \
                          left_on=['idpostulante','valor_educacion'], right_on = ['idpostulante','valor_educacion'])

post_educacion.head()

,idpostulante,nombre,estado,valor_educacion
0,NdJl,Posgrado,En Curso,15
1,axaPJ,Posgrado,En Curso,15
2,Z48mz,Posgrado,En Curso,15
3,a4zEZ,Posgrado,En Curso,15
4,6AWqO,Posgrado,En Curso,15


In [10]:
postulantes = pd.merge(post_educacion, post_gen_edad)
postulantes.head()

,idpostulante,nombre,estado,valor_educacion,fechanacimiento,sexo
0,NdJl,Posgrado,En Curso,15,1969-05-09,MASC
1,axaPJ,Posgrado,En Curso,15,1970-01-06,MASC
2,Z48mz,Posgrado,En Curso,15,1967-05-16,MASC
3,a4zEZ,Posgrado,En Curso,15,1976-04-10,MASC
4,6AWqO,Posgrado,En Curso,15,1970-05-21,MASC


## Ver postulaciones de secundario en curso y vistas hechas al respecto (vistas, postulaciones, área de trabajo).


In [13]:
postulantes_sec = postulantes.loc[(postulantes['nombre'] == 'Secundario') & (postulantes['estado'] == 'En Curso')]
postulantes_sec.head()

,idpostulante,nombre,estado,valor_educacion,fechanacimiento,sexo
54977,1qkm9z,Secundario,En Curso,5,1976-09-18,MASC
54978,YvjkMV,Secundario,En Curso,5,1975-06-18,FEM
54979,Y4oJ9w,Secundario,En Curso,5,1985-09-24,MASC
54980,5L5akv,Secundario,En Curso,5,1985-10-12,MASC
54981,6ZkmJR,Secundario,En Curso,5,1984-01-08,MASC


In [29]:
now = pd.Timestamp(dt.datetime.now())
#datetime.strptime(now, '%Y-%m-%d')
datetime.datetime.strptime(now, "%Y-%m-%d")
print(now)


TypeError: strptime() argument 1 must be str, not Timestamp

In [27]:
pd.to_datetime(postulantes_sec['fechapostulacion'])
postulantes_sec['age'] = (now - postulantes_sec['fechanacimiento']).astype('<m8[Y]')
print(postulantes_sec)

TypeError: unsupported operand type(s) for -: 'Timestamp' and 'str'